In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import dash
import numpy as np

# import plotly
import plotly.graph_objects as go
import plotly.express as px

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("All Months-Days Worked Data.csv")
file_path = "../Childrens Results/Results By Year/20230824- Children Database  - English Main.xlsx"
sheet_name = "Children Results "
children = pd.read_excel(file_path, sheet_name=sheet_name)
youth = pd.read_csv("20230905 - Combined Youth Data.csv")
youth_hours = pd.read_csv("2023 Youth Hours & Days-Days Worked Data.csv")
youth_stipends = pd.read_csv("2023 Youth Stipends-Stipends Main Data.csv")
session_tracker = pd.read_csv("Main - Combined Session Tracker - Consolidated.csv")
# session_tracker = pd.read_excel("20230905 - Online Session Tracker w IDs.xlsx", sheet_name="Sheet1")


/Users/jimmckeown/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
# Calculate 'Total - Hours', 'Total - Days', and 'Total - Stipend' columns
youth_hours['Total - Hours'] = youth_hours[['March - Hours', 'April - Hours', 'May - Hours', 'June - Hours', 'July - Hours']].sum(axis=1)
youth_hours['Total - Days'] = youth_hours[['March - Days', 'April - Days', 'May - Days', 'June - Days', 'July - Days']].sum(axis=1)
youth_hours['Total - Stipend'] = youth_hours[['March - Stipend', 'April - Stipend', 'May - Stipend', 'June - Stipend', 'July - Stipend']].sum(axis=1)


In [8]:
youth_hours['Days Through Aug'] = youth_hours[
    ['March - Days', 'April - Days', 'May - Days', 
     'June - Days', 'July - Days', 'August - Days']].sum(axis=1)

youth_hours['Days Through July'] = youth_hours[
    ['March - Days', 'April - Days', 'May - Days', 
     'June - Days', 'July - Days']].sum(axis=1)

#### Create LC Sessions Database

In [16]:
### The session tracker has the LC repeated many times as it's sessions per child. So here, I'm creating a df that has one record per each LC w/ total session columns per month.
lc_sessions = session_tracker.groupby('Employee ID').agg({
    'Total Sessions': 'sum',
    'Jan - March Total Sessions': 'sum',
    'April Total Sessions': 'sum',
    'May Total Sessions': 'sum',
    'June Total Sessions': 'sum',
    'July Total Sessions': 'sum',
    'August Total Sessions': 'sum'
}).reset_index()

# List of columns to convert
columns_to_convert = [
    'Jan - March Total Sessions', 'April Total Sessions', 'May Total Sessions', 'June Total Sessions', 'July Total Sessions', 'August Total Sessions'
]

# Convert each column to numeric
for col in columns_to_convert:
    session_tracker[col] = pd.to_numeric(session_tracker[col], errors='coerce')



,Employee ID,Total Sessions,Jan - March Total Sessions,April Total Sessions,May Total Sessions,June Total Sessions,July Total Sessions,August Total Sessions
0,213.0,300.0,40.0,44.0,56.0,41.0,2.0,117.0
1,425.0,451.0,0.0,32.0,92.0,81.0,101.0,145.0
2,427.0,674.0,154.0,69.0,153.0,72.0,63.0,163.0
3,429.0,394.0,17.0,37.0,97.0,67.0,79.0,97.0
4,430.0,471.0,155.0,36.0,116.0,60.0,65.0,97.0


In [18]:
### The session tracker has the LC repeated many times as it's sessions per child. So here, I'm creating a df that has one record per each LC w/ total session columns per month.
lc_sessions_2 = session_tracker.groupby(['Current LC', 'School']).agg({
    'Total Sessions': 'sum',
    'Jan - March Total Sessions': 'sum',
    'April Total Sessions': 'sum',
    'May Total Sessions': 'sum',
    'June Total Sessions': 'sum',
    'July Total Sessions': 'sum',
    'August Total Sessions': 'sum'
}).reset_index()

# List of columns to convert
columns_to_convert = [
    'Jan - March Total Sessions', 'April Total Sessions', 'May Total Sessions', 'June Total Sessions', 'July Total Sessions', 'August Total Sessions'
]

# Convert each column to numeric
for col in columns_to_convert:
    session_tracker[col] = pd.to_numeric(session_tracker[col], errors='coerce')

lc_sessions_2.head()

,Current LC,School,Total Sessions,Jan - March Total Sessions,April Total Sessions,May Total Sessions,June Total Sessions,July Total Sessions,August Total Sessions
0,Achuma Philani,Zukisa,113.0,0.0,0.0,25.0,23.0,23.0,42.0
1,Akhona Pita,Qaqawuli Godolozi,524.0,74.0,72.0,78.0,74.0,85.0,141.0
2,Alizwa Tsili,Sisonke,48.0,0.0,7.0,5.0,0.0,0.0,36.0
3,Alizwi Tsili,Sisonke,599.0,143.0,64.0,85.0,95.0,53.0,159.0
4,Anam Mabangula,Isaac Booi,320.0,33.0,27.0,70.0,24.0,28.0,138.0


### Merging Sessions & Hours

In [28]:
# merge sessions & youth hours
youth_pay_sessions = pd.merge(lc_sessions, youth_hours, on='Employee ID', how='inner')
# figure out which LCs weren't in merged / didn't have a match
# missing_from_session_tracker = session_tracker[~session_tracker['Employee ID'].isin(merged_inner['Employee ID'])]
# missing_from_youth_hours = youth_hours[~youth_hours['Employee ID'].isin(merged_inner['Employee ID'])]



# Merging in other info from youth database
youth_pay_sessions = youth_pay_sessions.merge(youth, on='Employee ID', how='left')



In [29]:
youth_pay_sessions['Sessions Thru July'] = youth_pay_sessions[
    ['Jan - March Total Sessions', 'April Total Sessions', 'May Total Sessions', 
     'June Total Sessions', 'July Total Sessions']
].sum(axis=1)

youth_pay_sessions['Sessions Thru July'] = youth_pay_sessions[
    ['Jan - March Total Sessions', 'April Total Sessions', 'May Total Sessions', 
     'June Total Sessions', 'July Total Sessions']
].sum(axis=1)


### Merging in a couple more columns from the session tracker

In [30]:
agg_df = session_tracker.groupby('Employee ID').agg({
    'Mcode': 'count',
    'Total Sessions': 'mean'
}).reset_index()

agg_df.columns = ['Employee ID', 'Number of Kids', 'Avg Sessions per Kid']

youth_pay_sessions = youth_pay_sessions.merge(agg_df, on='Employee ID', how='left')


In [31]:
youth.columns

Index(['Employee ID', 'Basic Link', 'Office Link', 'Payments Link',
       'First Names', 'Last Name', 'Employment Status', 'Mentor', 'Job Title',
       'Site Placement', 'Site Type', 'Start Date', 'End Date', 'Pay Rate',
       'Year of Service', 'Months Active', 'Reason for Leaving',
       'Future Employer', 'Gender', 'Race', 'Cell Phone Number', 'Email',
       'Street Number', 'Street Address', 'City or Town', 'Suburb/Township',
       'Postal Code', 'DOB', 'ID Type', 'Foreign ID Number', 'RSA ID Number',
       'Income Tax Number', 'Bank Name', 'Account type', 'Branch code',
       'Account number', 'January - Stipend', 'February - Stipend',
       'March - Stipend', 'April - Stipend', 'May - Stipend', 'June - Stipend',
       'Total Youth Stipend Paid', 'Attachment: ID Document',
       'Attachment: MOU/Contract', 'Attachment: Info Sheet',
       'Attachment: SARS Notice of registration',
       'Attachment: Change of Banking Form', 'Attachment: Resignation Letter',
       'Day

### This will print out a .csv with Per per Session and Sessions per Day

In [32]:
youth_pay_sessions['Pay per Session'] = youth_pay_sessions['Total - Stipend'] / youth_pay_sessions['Sessions Thru July']
youth_pay_sessions['Sessions per Day'] = youth_pay_sessions['Sessions Thru July'] / youth_pay_sessions['Days Through July']


youth_with_sessions = youth_pay_sessions[youth_pay_sessions['Sessions Thru July'] > 0]
export_df = youth_with_sessions[['Employee ID', 'LC Name', 'Mentor', 'Site Placement', 'Site Type', 'Number of Kids', 'Avg Sessions per Kid', 'Sessions Thru July', 'Days Through July', 'Sessions per Day', 'Total - Stipend', 'Pay per Session', 'Jan - March Total Sessions', 'April Total Sessions', 'May Total Sessions', 'June Total Sessions', 'July Total Sessions', 'August Total Sessions']].sort_values('Sessions per Day', ascending=False)
export_df.to_csv("pay_per_session.csv", index=False)


In [35]:
session_tracker.groupby('Current LC')['Mcode'].count().sort_values()

Current LC
Siphosethu Mxhasa     0
Jasmine Lodewyk       1
Alizwa Tsili          2
Khanyiswa Kuhlane     2
Phelelwe Nyoka        2
                     ..
Winnifred Walters    14
Nandipha Putela      14
Marizanne Mzangwa    15
Zikhona Gomo         15
Zintle Majola        15
Name: Mcode, Length: 161, dtype: int64